In [1]:
import numpy as np
import math
from scipy.stats import norm, gamma, poisson

In [3]:
SeattleA = np.array([[15,12,4],[11,17,4],[0,21,4],[0,0,5]])
SeattleA

array([[15, 12,  4],
       [11, 17,  4],
       [ 0, 21,  4],
       [ 0,  0,  5]])

In [21]:
#
# Process data
#

def process(DATA):
    hsA = np.sum(DATA,axis=0)
    isA = np.sum(DATA,axis=1)
    colA = np.array(range(1,len(hsA)+1))
    rowA = np.array(range(0,len(hsA)+1))
    
    xA = np.sum(isA*rowA) # Final size
    N = np.sum(hsA*colA) # Population size
    
    return {'hsA' : hsA, 'isA' : isA, 'colA' : colA, 'rowA' : rowA, 'xA' : xA, 'N' : N}

process(SeattleA)

{'N': 177,
 'colA': array([1, 2, 3]),
 'hsA': array([26, 50, 17]),
 'isA': array([31, 32, 25,  5]),
 'rowA': array([0, 1, 2, 3]),
 'xA': 97}

In [123]:
# Code for setting (local) thresholds in a household of size n.
# 
# Note local thresholds not required for households of size 1.
# Infection rate does not depend upon households size.
#

def thresH(n):
    thres = np.repeat(0.0,n-1)
    thres[0] = np.random.exponential(size=1,scale=1/(n-1))
    if n > 2:
        for i in range(1,n-1):
            thres[i] = thres[i-1] + np.random.exponential(size=1, scale=1/(n-i))
    
    return thres

thresH(5)

array([0.44338392, 0.51733957, 0.53504957, 1.15349319])

In [163]:
def House_epi(n,k,lambda_L):
    if n == 1:
        i = 1
        if k == 0:
            sev = 1
        if k > 0:
            sev = np.random.gamma(k, k, 1)
    
    if n > 1:
        t = thresH(n)
        if k == 0:
            q = np.repeat(1.0,n)
        if k > 0:
            q = np.random.gamma(size=n, shape=k, scale=1/k)
        t = np.append(t, 2*lambda_L*np.sum(q))
        
        i = 0
        test = 0
        while test == 0:
            i += 1
            if t[i-1] > (lambda_L*np.sum(q[0:i])):
                test = 1 
                sev = np.sum(q[0:i-1])
                
        return np.array([i,sev])
    
House_epi(1000,3,5)

array([1000.        , 1001.50275018])